In [1]:
%reload_ext autoreload
%autoreload 2

In [1]:
import pandas as pd
import sys
import os
sys.path.append(os.path.abspath('../'))
from src.hopsworks_connections import pull_data, upload_data

# Print the current working directory
current_working_directory = os.getcwd()
print("Current Working Directory:", current_working_directory)

/Users/davydsadovskyy/GitBackedShit/crypto-prediction/venv_crypto_prediction/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Current Working Directory: /Users/davydsadovskyy/GitBackedShit/crypto-prediction/notebooks


### Get a dataframe of the transformed ethereum OHLC data we made in 02_ohlc_feature_target_engineering.ipynb

In [2]:
eth_ohlc = pd.read_csv('../data/transformed/ohlc/ethereum.csv')
eth_ohlc

,date,open,high,low,close,volume,volume_eth,market_cap,tmw_avg_high_close,tmw_percent_increase,...,october,november,december,monday,tuesday,wednesday,thursday,friday,saturday,sunday
0,2015-11-15,0.8912,0.9215,0.8750,0.9064,4.118000e+05,458365.0,6.720000e+07,0.93685,0.032503,...,0,1,0,0,0,0,0,0,0,1
1,2015-11-16,0.9062,0.9447,0.8920,0.9290,6.209000e+05,676442.0,6.860000e+07,1.02000,0.089216,...,0,1,0,1,0,0,0,0,0,0
2,2015-11-17,0.9249,1.0300,0.9058,1.0100,1.100000e+06,1183690.0,7.220000e+07,1.00000,-0.010000,...,0,1,0,0,1,0,0,0,0,0
3,2015-11-18,0.9900,1.0100,0.9405,0.9900,6.811000e+05,691994.0,7.360000e+07,0.98275,-0.007377,...,0,1,0,0,0,1,0,0,0,0
4,2015-11-19,0.9887,1.0100,0.9375,0.9555,4.435000e+05,455866.0,7.280000e+07,0.94030,-0.016165,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3057,2024-04-17,3083.0000,3119.0000,2926.0000,2984.0000,1.550000e+10,5093979.0,3.722000e+11,3075.50000,0.029751,...,0,0,0,0,0,1,0,0,0,0
3058,2024-04-18,2986.0000,3087.0000,2960.0000,3064.0000,1.350000e+10,4453302.0,3.706000e+11,3101.00000,0.011932,...,0,0,0,0,0,0,1,0,0,0
3059,2024-04-19,3061.0000,3117.0000,2879.0000,3085.0000,1.740000e+10,5675824.0,3.746000e+11,3160.50000,0.023889,...,0,0,0,0,0,0,0,1,0,0
3060,2024-04-20,3085.0000,3166.0000,3025.0000,3155.0000,8.900000e+09,2884086.0,3.776000e+11,3168.50000,0.004261,...,0,0,0,0,0,0,0,0,1,0


### Upload this df to hopsworks feature store

In [5]:
# hopsworks requires datetime, when reading back from .csv, the datetime designation apparently goes away, along with nonnullable ints
eth_ohlc['date'] = pd.to_datetime(eth_ohlc['date'])

# Convert response binary columns to nullable integer type
columns = [
    'tmw_positive_percent_increase_binary',
    'tmw_1_0_percent_increase_binary',
    'tmw_1_25_percent_increase_binary',
    'tmw_1_5_percent_increase_binary',
    'tmw_1_75_percent_increase_binary',
    'tmw_2_0_percent_increase_binary',
    'tmw_2_25_percent_increase_binary',
    'tmw_2_5_percent_increase_binary',
    'tmw_2_75_percent_increase_binary',
    'tmw_3_0_percent_increase_binary'
]
for column in columns:
    eth_ohlc[column] = eth_ohlc[column].astype('Int64')

upload_data(eth_ohlc, 'eth_ohlc_transformed', 1)

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.
Feature Group created successfully, explore it at 
https://c.app.hopsworks.ai:443/p/582805/fs/578628/fg/744881


Uploading Dataframe: 100.00% |██████████| Rows 3062/3062 | Elapsed Time: 00:03 | Remaining Time: 00:00

Launching job: eth_ohlc_transformed_1_offline_fg_materialization
Job started successfully, you can follow the progress at 
https://c.app.hopsworks.ai/p/582805/jobs/named/eth_ohlc_transformed_1_offline_fg_materialization/executions


### Pull from the hopsworks to check if the df is fine - theres a several minute lag time between uploading this data and being able to pull it...

In [9]:
hopsworks_eth_ohlc = pull_data('eth_ohlc_transformed', 1, 'eth_ohlc_transformed_view', 1)
hopsworks_eth_ohlc

Connection closed.
Connected. Call `.close()` to terminate connection gracefully.




To ensure compatibility please install the latest bug fix release matching the minor version of your backend (3.7) by running 'pip install hopsworks==3.7.*'



Logged in to project, explore it here https://c.app.hopsworks.ai:443/p/582805
Connected. Call `.close()` to terminate connection gracefully.
Feature view already existed. Skip creation.
Finished: Reading data from Hopsworks, using ArrowFlight (1.53s) 


,date,open,high,low,close,volume,volume_eth,market_cap,tmw_avg_high_close,tmw_percent_increase,...,october,november,december,monday,tuesday,wednesday,thursday,friday,saturday,sunday
806,2015-11-15 00:00:00+00:00,0.891200,0.921500,0.875000,0.906400,4.118000e+05,4.583650e+05,6.720000e+07,0.936850,0.032503,...,0,1,0,0,0,0,0,0,0,1
1225,2015-11-16 00:00:00+00:00,0.906200,0.944700,0.892000,0.929000,6.209000e+05,6.764420e+05,6.860000e+07,1.020000,0.089216,...,0,1,0,1,0,0,0,0,0,0
2432,2015-11-17 00:00:00+00:00,0.924900,1.030000,0.905800,1.010000,1.100000e+06,1.183690e+06,7.220000e+07,1.000000,-0.010000,...,0,1,0,0,1,0,0,0,0,0
1968,2015-11-18 00:00:00+00:00,0.990000,1.010000,0.940500,0.990000,6.811000e+05,6.919940e+05,7.360000e+07,0.982750,-0.007377,...,0,1,0,0,0,1,0,0,0,0
859,2015-11-19 00:00:00+00:00,0.988700,1.010000,0.937500,0.955500,4.435000e+05,4.558660e+05,7.280000e+07,0.940300,-0.016165,...,0,1,0,0,0,0,1,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2751,2024-04-18 00:00:00+00:00,2986.000000,3087.000000,2960.000000,3064.000000,1.350000e+10,4.453302e+06,3.706000e+11,3101.000000,0.011932,...,0,0,0,0,0,0,1,0,0,0
2793,2024-04-19 00:00:00+00:00,3061.000000,3117.000000,2879.000000,3085.000000,1.740000e+10,5.675824e+06,3.746000e+11,3160.500000,0.023889,...,0,0,0,0,0,0,0,1,0,0
2705,2024-04-20 00:00:00+00:00,3085.000000,3166.000000,3025.000000,3155.000000,8.900000e+09,2.884086e+06,3.776000e+11,3168.500000,0.004261,...,0,0,0,0,0,0,0,0,1,0
951,2024-04-21 00:00:00+00:00,3153.000000,3191.000000,3120.000000,3146.000000,8.200000e+09,2.603957e+06,3.867000e+11,3215.178412,0.021516,...,0,0,0,0,0,0,0,0,0,1
